In [34]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import json
from collections import Counter

In [35]:
# Function to save data as JSON without escape characters
def save_json_without_escape(filepath, dataframe):
    with open(filepath, 'w', encoding="utf-8") as f:
        json.dump(dataframe.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

In [36]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
with open('dataset/haveMD_302.json', 'r', encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Categories of interest
categories_of_interest = {"Generic policy", "Reporting mechanism", "Scope of practice", "User guideline"}

# Binary encoding for each category
for category in categories_of_interest:
    df[category] = df['SecurityPolicy_content_category'].apply(lambda x: 1 if category in x else 0)

# Create train, validation, and test splits for each category
for category in categories_of_interest:
    # Filter data where the current category is present (binary = 1)
    category_df = df.copy()
    
    if not category_df.empty:
        # Stratified train-test split (70-30)
        train_df, temp_df = train_test_split(
            category_df, test_size=0.3, stratify=category_df[category]
        )

        # Split temp_df into validation and test sets (20-10 split)
        validation_df, test_df = train_test_split(
            temp_df, test_size=(1/3), stratify=temp_df[category]
        )

        # Drop binary columns from the splits
        train_df = train_df.drop(columns=categories_of_interest)
        validation_df = validation_df.drop(columns=categories_of_interest)
        test_df = test_df.drop(columns=categories_of_interest)

        # Save splits to JSON files
        save_json_without_escape(f'dataset/for_modeling/train_{category}.json', train_df)
        save_json_without_escape(f'dataset/for_modeling/validation_{category}.json', validation_df)
        save_json_without_escape(f'dataset/for_modeling/test_{category}.json', test_df)

        print(f"Splits created and saved for category: {category}")
    else:
        print(f"No data found for category: {category}")


Splits created and saved for category: Generic policy
Splits created and saved for category: Reporting mechanism
Splits created and saved for category: Scope of practice
Splits created and saved for category: User guideline


In [41]:
def read_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return pd.DataFrame(json.load(f))

what = "Generic policy"
train_df = read_json(f'dataset/for_modeling/train_{what}.json')
validation_df = read_json(f'dataset/for_modeling/validation_{what}.json')
test_df = read_json(f'dataset/for_modeling/test_{what}.json')

datasets = {"Train": train_df, "Validation": validation_df, "Test": test_df}

# Process each set
for dataset_name, dataset in datasets.items():
    # Total number of packages
    total_packages = len(dataset)
    print(f"\n{dataset_name} Dataset: Total Packages = {total_packages}")
    
    # Count packages for each category
    category_counts = Counter(
        category for categories in dataset['SecurityPolicy_content_category'] for category in categories
    )
    print(f"Category Counts in {dataset_name} Dataset:")
    for category, count in category_counts.items():

        print(f"{category}: {count}")
    
    


Train Dataset: Total Packages = 211
Category Counts in Train Dataset:


TypeError: unhashable type: 'dict'